In [1]:
import pandas as pd
import numpy as np
import copy
import os
import re
from netaddr import *
import ipaddress
import time

/home/k.goto/.pyenv/versions/3.6.7/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/k.goto/.pyenv/versions/3.6.7/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


### 経路情報 10月30日 00:00時点
### RIR stats (apnicからjpnicへ割り振られた分のIP)を100行ずつ読み，その中で経路情報を200行ずつ読む

In [2]:
# CIDR表記から含まれるipアドレスのリストを得る関数
def cidr2ip(entry):
    x = [str(ip) for ip in ipaddress.IPv4Network(entry)]
    return np.array(x)

TextFileReader を for でループさせると、ファイルの中身を指定した行数ごとに DataFrame として読み取る。TextFileReader は 現時点での読み取り位置 (ポインタ) を覚えており、ファイルの中身をすべて読み取るとループが終了する。

### 最初からcidrを展開してset型で比較する方法

In [7]:
# route information
fr_file = "fullroute_20191030.0000"
dtyp = {"Peer ASN (ASN of the monitor)": "int16"}
rcc06_df_reader = pd.read_csv(fr_file, delimiter = "|", names = ("BGP Protocol", "timestamp", "W/A/B", "Peer IP(address of the monitor)", "Peer ASN (ASN of the monitor)", "Prefix", "ASPath", "Origin Protocol"), dtype=dtyp, chunksize = 200)

# jp addresses
jp_file = "jp_cidr"
jp_df_reader = pd.read_csv(jp_file, delimiter = "\t", names = ("cc (JP)", "IP block"), chunksize = 100)

In [6]:
start = time.time()
j = 0
for rcc06_chunk in rcc06_df_reader:
#     if j == 1:
#         break

    # 不要なカラムは消す
    del rcc06_chunk["BGP Protocol"], rcc06_chunk["timestamp"], rcc06_chunk["W/A/B"], \
        rcc06_chunk["Peer IP(address of the monitor)"], rcc06_chunk["Peer ASN (ASN of the monitor)"], \
        rcc06_chunk["ASPath"], rcc06_chunk["Origin Protocol"]
        
    # bv_ip_blocks = np.array([cidr2ip(entry) for entry in rcc06_chunk["Prefix"] if "/24" in entry]).ravel()じゃないのか?
    bv_ip_blocks = [cidr2ip(entry) for entry in rcc06_chunk["Prefix"] if "/24" in entry]
    print(j)
    # for optimisation
    bv_all_ip = []
    bv_all_ip_append = bv_all_ip.append
    
    for ips in bv_ip_blocks:
        for ip in ips:
            bv_all_ip_append(ip)
    
    b = set(bv_all_ip)
    
    i = 0
    # ここを1回作ったら保存しておくことはできないか...?
    for jp_chunk in jp_df_reader:
#         if i == 5:
#             break

        # jp_ip_blocks = np.array([cidr2ip(entry) for entry in jp_chunk["IP block"] if "/24" in entry]).ravel()じゃないのか?
        jp_ip_blocks = [cidr2ip(entry) for entry in jp_chunk["IP block"] if "/24" in entry]
    
        # for optimisation
        all_ip = []
        all_ip_append = all_ip.append
        
        for ips in jp_ip_blocks:
            for ip in ips:
                   all_ip.append(ip)

        del jp_ip_blocks

        a = set(all_ip)
        
        # bとaの共通集合をとり，これをbからのぞく...残りを次のbとしてaのforがつづく
        com = b & a
        b = b - com
        
#         i += 1
    
    print(f"{len(b)} in total; {list(b)[:5]}...\n")
    j += 1
    
elapsed = time.time() - start

0
11264 in total; ['1.0.221.80', '1.0.128.75', '1.0.0.110', '1.0.201.34', '1.1.110.164']...

1
9728 in total; ['1.1.253.155', '1.2.160.94', '1.2.4.234', '1.2.4.239', '1.1.115.195']...

2
9728 in total; ['1.2.180.25', '1.2.239.210', '1.4.159.44', '1.4.197.158', '1.2.238.177']...

3
4096 in total; ['1.6.161.93', '1.6.161.183', '1.6.66.227', '1.6.73.200', '1.6.146.221']...

4
10496 in total; ['1.10.166.91', '1.6.228.10', '1.10.139.97', '1.10.131.155', '1.10.72.173']...

5
14848 in total; ['1.10.251.65', '1.10.209.223', '1.10.222.156', '1.10.205.196', '1.10.171.16']...

6
11520 in total; ['1.20.81.17', '1.18.135.247', '1.20.91.62', '1.18.122.114', '1.20.87.14']...

7
14080 in total; ['1.20.163.132', '1.20.188.112', '1.20.168.89', '1.20.164.166', '1.20.144.190']...

8
15360 in total; ['1.20.245.56', '1.22.40.108', '1.20.226.54', '1.22.37.250', '1.22.24.118']...

9
14848 in total; ['1.22.102.185', '1.22.55.45', '1.22.85.46', '1.22.54.78', '1.22.77.144']...

10
14592 in total; ['1.22.155.122'

AddressValueError: Expected 4 octets in '2001:1c00::'

### 参考: /24のみ展開して表示

In [22]:
for jp_chunk in jp_df_reader:

        # /24以外は無視するよう変更...
        jp_ip_blocks = np.array([cidr2ip(entry) for entry in jp_chunk["IP block"] if "/24" in entry])
        
        all_ip = []
        all_ip_append = all_ip.append
        for ips in jp_ip_blocks:
            for ip in ips:
                   all_ip_append(ip)

        del jp_ip_blocks

        a = set(all_ip)
        print(list(a)[:5])

[]
[]
['61.47.3.203', '61.47.0.18', '61.47.21.31', '61.47.15.110', '58.145.228.59']
['61.47.56.83', '61.47.66.232', '61.47.53.5', '61.47.66.230', '61.47.46.142']
['103.11.221.68', '103.11.221.105', '103.14.149.40', '103.11.221.39', '103.11.221.75']
['103.48.31.76', '103.28.73.115', '103.48.31.81', '103.48.31.197', '103.47.2.175']
['103.61.151.78', '103.91.184.189', '103.100.189.10', '103.61.151.150', '103.59.89.60']
['103.144.142.117', '103.135.244.165', '103.118.123.21', '103.204.13.44', '103.142.127.212']
['103.241.104.190', '103.246.39.87', '103.241.104.115', '103.226.54.150', '103.245.224.69']
['103.247.181.86', '103.247.181.208', '103.246.232.179', '103.248.251.126', '103.248.251.194']
[]
['116.92.15.78', '116.92.132.48', '116.92.3.172', '116.92.15.233', '116.92.15.23']
[]
[]
['134.42.227.58', '134.42.227.235', '134.42.227.176', '134.42.227.168', '134.42.227.221']
['148.59.57.90', '149.234.190.189', '149.234.188.65', '149.234.190.117', '148.59.57.3']
['160.238.1.112', '163.47.24.8

### netdaddrのIPSetsのremoveを使って比較したのちcidrを展開する方法

In [6]:
# route information
fr_file = "fullroute_20191030.0000"
dtyp = {"Peer ASN (ASN of the monitor)": "int16"}
rcc06_df = pd.read_csv(fr_file, delimiter = "|", names = ("BGP Protocol", "timestamp", "W/A/B", "Peer IP(address of the monitor)", "Peer ASN (ASN of the monitor)", "Prefix", "ASPath", "Origin Protocol"), dtype=dtyp)
# jp addresses
jp_file = "jp_cidr"
jp_df = pd.read_csv(jp_file, delimiter = "\t", names = ("cc (JP)", "IP block"))
# unallocated jp addresses
jpnic_pooled = "unallocated-jpnic-20191101.txt"
pooled_df = pd.read_csv(jpnic_pooled, header = None, names = ("IP block",))

In [5]:
suspects = set()

a = 0
for rcc_row in rcc06_df.itertuples():
    # 127316以降はIPv6のため，一旦無視 (20/11/2019)
#     if a == 127316:
#         break 
    if a == 500:
        break
        
    n1 = IPSet()
    n1.add(IPNetwork(rcc_row[6]))
    
    b = 0
    for jp_row in jp_df.itertuples():
#         if b == 300:
#             break
        
        n2 = IPSet()
        n2.add(IPNetwork(jp_row[2]))
        n1.remove(n2.iprange())
        
        b += 1
    suspects.add(str(n1))
    a += 1
    
s_raw = list(suspects)
# 本来は正規表現でcidr部分だけを抽出したい
s_blocks = [x.replace("IPSet(['", "").replace("'])", "") for i, x in enumerate(s_raw) if x != "IPSet([])"]
suspect_ips = [cidr2ip(block) for block in s_blocks]

sus = []
sus_append = sus.append
        
for ips in suspect_ips:
    for ip in ips:
        sus_append(ip)
            
print(f"Num of suspects: {len(sus)}\nContents: {sus[:5]}...")

Num of suspects: 226560
Contents: ['1.0.128.0', '1.0.128.1', '1.0.128.2', '1.0.128.3', '1.0.128.4']...


In [31]:
final_sus = set()

for block in s_blocks:
    n1 = IPSet()
    n1.add(IPNetwork(block))
    for p_block in pooled_df.itertuples():
        n2 = IPSet()
        # ここハードコーディングなのであとで直さないと使い物にならない
        if ("-" in (p_block[1])):
            s1 = IPSet()
            s1.add(IPRange("211.120.252.0", "211.120.254.255"))
            # n1とn2の共通集合
            intersection = n1 & s1
        else: 
            n2.add(IPNetwork(str(p_block[1])))
            # n1とn2の共通集合
            intersection = n1 & n2
        final_sus.add(str(intersection))

sus_raw = list(final_sus)
# 本来は正規表現でcidr部分だけを抽出したい
sus_blocks = [x.replace("IPSet(['", "").replace("'])", "") for i, x in enumerate(sus_raw) if x != "IPSet([])"]
print(sus_blocks)

[]

### df読み込み

In [209]:
# route information
fr_file = "fullroute_20191030.0000"
dtyp = {"Peer ASN (ASN of the monitor)": "int16"}
rcc06_df = pd.read_csv(fr_file, delimiter = "|", names = ("BGP Protocol", "timestamp", "W/A/B", "Peer IP(address of the monitor)", "Peer ASN (ASN of the monitor)", "Prefix", "ASPath", "Origin Protocol"), dtype=dtyp)
# jp addresses
jp_file = "jp_cidr"
jp_df = pd.read_csv(jp_file, delimiter = "\t", names = ("cc (JP)", "IP block"))
# unallocated jp addresses
jpnic_pooled = "unallocated-jpnic-20191101.txt"
pooled_df = pd.read_csv(jpnic_pooled, header = None, names = ("IP block",))

### まず経路情報中に出現するアドレスのうち，jpnicが割り振られたものをとる; つまり共通部分をとる

In [149]:
scope_raw = set()

for rcc_row in rcc06_df[561500:561500 + 500][["Prefix"]].itertuples():
        
    n1 = IPSet()
    n1.add(IPNetwork(rcc_row[1]))
    
    for jp_row in jp_df.itertuples():
        n2 = IPSet()
        n2.add(IPNetwork(jp_row[2]))
        # n1とn2の共通部分
        intersection = n1 & n2
        scope_raw.add(str(intersection))
        
scope = list(scope_raw)
scope_blocks = [x.replace("IPSet(['", "").replace("'])", "") for i, x in enumerate(scope_raw) if x != "IPSet([])"]
print(f"{len(scope_blocks)} blocks are in scope")

81 blocks are in scope


### 次にその中でもEUへ未割り当てのものをとる; 共通部分をとる

In [150]:
sus_raw = set()

for block in scope_blocks:
    n1 = IPSet()
    n1.add(IPNetwork(block))
    for p_block in pooled_df.itertuples():
        n2 = IPSet()
        # ここハードコーディングなのであとで直す
        if ("-" in (p_block[1])):
            s1 = IPSet()
            s1.add(IPRange("211.120.252.0", "211.120.254.255"))
            # n1とs1の共通部分
            intersection = n1 & s1
        else:
            n2.add(IPNetwork(str(p_block[1])))
            # n1とn2の共通部分
            intersection = n1 & n2
        sus_raw.add(str(intersection))

sus = list(sus_raw)
# 本来は正規表現でcidr部分だけを抽出したい
sus_blocks = [x.replace("IPSet(['", "").replace("'])", "") for i, x in enumerate(sus) if x != "IPSet([])"]
print(f"{len(sus_blocks)} blocks found")

0 blocks found


### /8ごとに経路とjpを比較し，共通部分をとる 保存処理を書いていないので次回以降実装

In [ ]:
rcc06_df = pd.read_csv(fr_file, delimiter = "|", names = ("BGP Protocol", "timestamp", "W/A/B", "Peer IP(address of the monitor)", "Peer ASN (ASN of the monitor)", "Prefix", "ASPath", "Origin Protocol"), dtype=dtyp)

del rcc06_df["BGP Protocol"], rcc06_df["timestamp"], rcc06_df["W/A/B"], \
    rcc06_df["Peer IP(address of the monitor)"], rcc06_df["Peer ASN (ASN of the monitor)"], \
    rcc06_df["ASPath"], rcc06_df["Origin Protocol"]

start = time.time()

# /8ごとにjp_dfを切り取って回る
for i in range(1, 256):
    head = str(i) + "{1}\."
    jp_section = jp_df.applymap(lambda x: bool(re.match(head, x)))
    jp_sub_table = jp_df[jp_section]["IP block"].dropna()
    
    rcc_section = rcc06_df.applymap(lambda x: bool(re.match(head, x)))
    rcc_sub_table = rcc06_df[rcc_section]["Prefix"].dropna()
    
    scope_raw = set()
    for rcc_row in rcc_sub_table:
        n1 = IPSet()
        n1.add(IPNetwork(rcc_row))
    
        for jp_row in jp_sub_table:
            n2 = IPSet()
            n2.add(IPNetwork(jp_row))
            # n1とn2の共通部分
            intersection = n1 & n2
            scope_raw.add(str(intersection))
            
    scope = list(scope_raw)
    scope_blocks = [x.replace("IPSet(['", "").replace("'])", "") for i, x in enumerate(scope_raw) if x != "IPSet([])"]
    print(f"{len(scope_blocks)} blocks are in scope")

elapsed = time.time() - start

print(f"elapsed time: {elapsed/(60*60)} [h]")

285 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
4 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
32 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
1 blocks are in scope
2 blocks are in scope
0 blocks are in scope
0 blocks are in scope
200 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
0 blocks are in scope
89 blocks are in scope
0 blocks are in scope
0 blocks are in scope
3 blocks are in scope


In [183]:
pd.options.display.max_rows = 3000

rcc06_dfのindex 127316 - 210960はIPv6 これ以外にもありそう

In [117]:
rcc06_df[127315:210961][["Prefix"]]

,Prefix
127315,32.0.0.0/8
127316,2001::/32
127317,2001::/32
127318,2001::/32
127319,2001:4:112::/48
...,...
210956,240f::/24
210957,240f:100::/24
210958,240f:100::/24
210959,240f:100::/24


## 元のデータフレーム

#### 経路情報

In [181]:
rcc06_df

,BGP Protocol,timestamp,W/A/B,Peer IP(address of the monitor),Peer ASN (ASN of the monitor),Prefix,ASPath,Origin Protocol
0,TABLE_DUMP2,10/30/19 00:00:00,B,202.249.2.169,2497,1.0.0.0/24,2497 13335,IGP
1,TABLE_DUMP2,10/30/19 00:00:00,B,202.249.2.185,25152,1.0.0.0/24,25152 2914 13335,IGP
2,TABLE_DUMP2,10/30/19 00:00:00,B,202.249.2.20,4777,1.0.0.0/24,4777 13335,IGP
3,TABLE_DUMP2,10/30/19 00:00:00,B,202.249.2.169,2497,1.0.4.0/22,2497 4826 38803 56203,IGP
4,TABLE_DUMP2,10/30/19 00:00:00,B,202.249.2.185,25152,1.0.4.0/22,25152 6939 4826 38803 56203,IGP
...,...,...,...,...,...,...,...,...
2571504,TABLE_DUMP2,10/30/19 00:00:00,B,202.249.2.185,25152,223.255.253.0/24,25152 2914 4134 58519,IGP
2571505,TABLE_DUMP2,10/30/19 00:00:00,B,202.249.2.20,4777,223.255.253.0/24,4777 2516 4134 58519,IGP
2571506,TABLE_DUMP2,10/30/19 00:00:00,B,202.249.2.169,2497,223.255.254.0/24,2497 7473 3758 55415,IGP
2571507,TABLE_DUMP2,10/30/19 00:00:00,B,202.249.2.185,25152,223.255.254.0/24,25152 6939 4657 55415 55415 55415 55415 55415 ...,IGP


#### jpnicに割り振られたもの

In [72]:
jp_df

,cc (JP),IP block
0,JP,1.0.16.0/20
1,JP,1.0.64.0/18
2,JP,1.1.64.0/18
3,JP,1.5.0.0/16
4,JP,1.21.0.0/16
...,...,...
2852,JP,223.223.160.0/21
2853,JP,223.223.208.0/21
2854,JP,223.223.224.0/19
2855,JP,223.252.64.0/19


#### jpnicに割り振られたが，EUへは未割り当てのもの

In [147]:
pooled_df

,IP block
0,61.122.4.0/22
1,61.122.8.0/21
2,61.122.20.0/22
3,61.122.24.0/21
4,61.122.96.0/22
5,61.122.100.0/22
6,61.122.104.0/21
7,61.122.192.0/20
8,61.125.176.0/22
9,61.125.180.0/22
